# The Following 2 Blocks are scripts to synthesize english number datasets the first is an optimal script while the second is nonoptimal

In [ ]:
# import os
# import cv2
# import numpy as np
# import random
# from PIL import Image, ImageDraw, ImageFont

# # --- CONFIGURATION ---
# OUTPUT_DIR = "dataset"
# SAMPLES_PER_DIGIT = 200 # How many images per number to generate
# IMG_SIZE = (64, 64)      # Canvas size (larger than target to allow rotation/cropping)
# FONT_PATH = "arial.ttf"  # <--- COPY A FONT FILE HERE!
# FONT_SIZE = 45

# # Create folders 0-9
# if not os.path.exists(OUTPUT_DIR):
#     os.makedirs(OUTPUT_DIR)
# for i in range(10):
#     os.makedirs(os.path.join(OUTPUT_DIR, str(i)), exist_ok=True)

# def apply_augmentations(img_pil):
#     """
#     Takes a clean PIL image and ruins it to look like a scanned ID.
#     """
#     # Convert to NumPy for OpenCV processing
#     img = np.array(img_pil) 
    
#     # 1. Random Rotation (-10 to 10 degrees)
#     angle = random.uniform(-10, 10)
#     h, w = img.shape
#     M = cv2.getRotationMatrix2D((w/2, h/2), angle, 1)
#     img = cv2.warpAffine(img, M, (w, h), borderValue=0) # Black border

#     # 2. Gaussian Blur (Simulate out-of-focus camera)
#     if random.random() > 0.5:
#         k = random.choice([3, 5])
#         img = cv2.GaussianBlur(img, (k, k), 0)

#     # 3. Noise (Simulate sensor grain)
#     noise = np.random.randint(0, 50, (h, w), dtype='uint8')
#     # Add noise only to non-black areas mostly, or just add it overall
#     img = cv2.add(img, noise)

#     # 4. Erosion/Dilation (Simulate ink bleeding or thin print)
#     if random.random() > 0.5:
#         kernel = np.ones((2,2), np.uint8)
#         if random.choice([True, False]):
#             img = cv2.erode(img, kernel, iterations=1)
#         else:
#             img = cv2.dilate(img, kernel, iterations=1)

#     return img

# def generate_dataset():
#     print(f"Generating {SAMPLES_PER_DIGIT} images per digit...")
    
#     try:
#         font = ImageFont.truetype(FONT_PATH, FONT_SIZE)
#     except IOError:
#         print("ERROR: Font file not found! Please put 'arial.ttf' in this folder.")
#         return

#     for digit in range(10):
#         print(f"Processing digit: {digit}")
#         for i in range(SAMPLES_PER_DIGIT):
#             # 1. Create a blank black image
#             img_pil = Image.new('L', IMG_SIZE, color=0)
#             draw = ImageDraw.Draw(img_pil)
            
#             # 2. Draw the digit in white centered(ish)
#             # We add random offset so the number isn't always perfectly in the middle
#             text = str(digit)
#             # Get text bounding box to center it
#             bbox = draw.textbbox((0, 0), text, font=font)
#             text_w = bbox[2] - bbox[0]
#             text_h = bbox[3] - bbox[1]
            
#             x = (IMG_SIZE[0] - text_w) / 2 + random.randint(-5, 5)
#             y = (IMG_SIZE[1] - text_h) / 2 + random.randint(-5, 5)
            
#             draw.text((x, y), text, font=font, fill=255)
            
#             # 3. Apply the "Reality" effects
#             final_img = apply_augmentations(img_pil)
            
#             # 4. Save
#             save_path = os.path.join(OUTPUT_DIR, str(digit), f"{digit}_{i}.png")
#             cv2.imwrite(save_path, final_img)

#     print("Done! You now have a dataset.")

# if __name__ == "__main__":
#     generate_dataset()

In [54]:
# import os
# import cv2
# import numpy as np
# import random
# from PIL import Image, ImageDraw, ImageFont

# # --- CONFIGURATION ---
# OUTPUT_DIR = "dataset_mixed"
# SAMPLES_PER_DIGIT = 400  # Total images per digit
# IMG_SIZE = (64, 64)

# # List your fonts here. Ensure these files are in the folder!
# FONTS = ["arial.ttf", "Sitka.ttc"] 
# FONT_SIZE = 45

# # Create output folders
# if not os.path.exists(OUTPUT_DIR):
#     os.makedirs(OUTPUT_DIR)
# for i in range(10):
#     os.makedirs(os.path.join(OUTPUT_DIR, str(i)), exist_ok=True)

# def add_shadow(img):
#     """Simulates uneven lighting."""
#     h, w = img.shape
#     top_left = random.uniform(0.5, 1.0)
#     bot_right = random.uniform(0.5, 1.0)
#     X, Y = np.meshgrid(np.arange(w), np.arange(h))
#     mask = top_left + (bot_right - top_left) * (X / w)
#     img = img.astype('float32') * mask
#     return img.astype('uint8')

# def break_character(img):
#     """Simulates scratches and missing chunks."""
#     h, w = img.shape
    
#     # Random scratch line
#     if random.random() > 0.5:
#         num_scratches = random.randint(1, 3)
#         for _ in range(num_scratches):
#             x1, y1 = random.randint(0, w), random.randint(0, h)
#             x2, y2 = random.randint(0, w), random.randint(0, h)
#             cv2.line(img, (x1, y1), (x2, y2), 0, random.randint(1, 3))

#     # Random noise chunks missing
#     if random.random() > 0.5:
#         noise = np.zeros((h, w), dtype='uint8')
#         cv2.randn(noise, 0, 255)
#         _, holes = cv2.threshold(noise, 200, 255, cv2.THRESH_BINARY)
#         img = cv2.subtract(img, holes)
        
#     return img

# def apply_defects(img_pil):
#     img = np.array(img_pil)
    
#     # 1. Perspective Warp
#     h, w = img.shape
#     src_points = np.float32([[0,0], [w,0], [0,h], [w,h]])
#     dst_points = np.float32([
#         [random.randint(0, 5), random.randint(0, 5)],
#         [w - random.randint(0, 5), random.randint(0, 5)],
#         [random.randint(0, 5), h - random.randint(0, 5)],
#         [w - random.randint(0, 5), h - random.randint(0, 5)]
#     ])
#     M = cv2.getPerspectiveTransform(src_points, dst_points)
#     img = cv2.warpPerspective(img, M, (w, h))

#     # 2. Break Character
#     img = break_character(img)

#     # 3. Blur
#     if random.random() > 0.3:
#         k = random.choice([3, 5])
#         img = cv2.GaussianBlur(img, (k, k), 0)

#     # 4. Shadow
#     if random.random() > 0.4:
#         img = add_shadow(img)

#     # 5. Salt & Pepper Noise
#     noise_prob = 0.02
#     thres = 1 - noise_prob
#     rdn = np.random.random(img.shape)
#     img[rdn < noise_prob] = 0
#     img[rdn > thres] = 255

#     return img

# def generate_dataset():
#     print(f"Generating {SAMPLES_PER_DIGIT} images per digit using fonts: {FONTS}")
    
#     # Verify fonts exist
#     loaded_fonts = []
#     for f_name in FONTS:
#         try:
#             loaded_fonts.append(ImageFont.truetype(f_name, FONT_SIZE))
#         except IOError:
#             print(f"WARNING: Could not load {f_name}. Skipping.")
    
#     if not loaded_fonts:
#         print("ERROR: No fonts found! Please copy .ttf files to this folder.")
#         return

#     for digit in range(10):
#         print(f"Processing digit: {digit}")
#         for i in range(SAMPLES_PER_DIGIT):
#             # 1. Randomly choose a font
#             font = random.choice(loaded_fonts)
            
#             # 2. Draw Text
#             img_pil = Image.new('L', IMG_SIZE, color=0)
#             draw = ImageDraw.Draw(img_pil)
            
#             text = str(digit)
#             bbox = draw.textbbox((0, 0), text, font=font)
#             text_w, text_h = bbox[2] - bbox[0], bbox[3] - bbox[1]
#             x = (IMG_SIZE[0] - text_w) / 2 + random.randint(-8, 8)
#             y = (IMG_SIZE[1] - text_h) / 2 + random.randint(-8, 8)
            
#             draw.text((x, y), text, font=font, fill=255)
            
#             # 3. Apply Defects
#             final_img = apply_defects(img_pil)
            
#             # 4. Save
#             save_path = os.path.join(OUTPUT_DIR, str(digit), f"{digit}_{i}.png")
#             cv2.imwrite(save_path, final_img)

#     print(f"Done! Created {SAMPLES_PER_DIGIT * 10} images in '{OUTPUT_DIR}'")

# if __name__ == "__main__":
#     generate_dataset()

In [55]:
from sklearn.neural_network import MLPClassifier  # MLP is an NN
from sklearn import svm
import numpy as np
import argparse
import imutils  # If you are unable to install this library, ask the TA; we only need this in extract_hsv_histogram.
import cv2
import os
import random

from sklearn.svm import LinearSVC
# Depending on library versions on your system, one of the following imports 
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split

In [56]:
path_to_dataset = "./dataset_mixed"

target_img_size = (32, 32) # fix image size because classification algorithms THAT WE WILL USE HERE expect that
# We are going to fix the random seed to make our experiments reproducible 
# since some algorithms use pseudorandom generators
random_seed = 42  
random.seed(random_seed)
np.random.seed(random_seed)

In [57]:
def extract_hog_features(img):
    """
    TODO
    You won't implement anything in this function. You just need to understand it 
    and understand its parameters (i.e win_size, cell_size, ... etc)
    """
    img = cv2.resize(img, target_img_size)
    win_size = (32, 32)
    cell_size = (4, 4)
    block_size_in_cells = (2, 2)
    
    block_size = (block_size_in_cells[1] * cell_size[1], block_size_in_cells[0] * cell_size[0])
    block_stride = (cell_size[1], cell_size[0])
    nbins = 9  # Number of orientation bins
    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins)
    h = hog.compute(img)
    h = h.flatten()
    return h.flatten()

In [ ]:
def load_dataset(feature_set='hog'):
    features = []
    labels = []
    img_filenames = os.listdir(path_to_dataset)
    total_processed = 0
    for folder in img_filenames:    
        total_images += len(img_filenames) * len(os.listdir(os.path.join(path_to_dataset, folder)))   # 10 folders * 400 images each = 4000

    for i, fn in enumerate(img_filenames):
        NumberInnerPath = os.path.join(path_to_dataset, str(i))
        NumberInnerPath = os.listdir(NumberInnerPath)
        for idx, img_filename in enumerate(NumberInnerPath):
            if img_filename.split('.')[-1] != 'png':
                continue

            label = str(i)
            labels.append(label)

            # Build the full file path
            full_path = os.path.join(path_to_dataset, str(i), img_filename)
            img = cv2.imread(full_path)
            features.append(extract_hog_features(img))
            
            total_processed += 1
            
            # show an update every 50 images
            if total_processed > 0 and total_processed % 50 == 0:
                print("[INFO] processed {}/{} images".format(total_processed, total_images))
        
    return features, labels        

In [59]:
def train_SVM():
    SVMclassifier=svm.LinearSVC(random_state=random_seed)
    features, labels = load_dataset()
    train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.4, random_state=random_seed)
    SVMclassifier.fit(train_features,train_labels)
    accuracy=SVMclassifier.score(test_features,test_labels)
    print("SVM Accuracy: ", accuracy)

In [60]:
train_SVM()

[INFO] processed 50/2000 images
[INFO] processed 100/2000 images
[INFO] processed 150/2000 images
[INFO] processed 200/2000 images
[INFO] processed 250/2000 images
[INFO] processed 300/2000 images
[INFO] processed 350/2000 images
[INFO] processed 400/2000 images
[INFO] processed 450/2000 images
[INFO] processed 500/2000 images
[INFO] processed 550/2000 images
[INFO] processed 600/2000 images
[INFO] processed 650/2000 images
[INFO] processed 700/2000 images
[INFO] processed 750/2000 images
[INFO] processed 800/2000 images
[INFO] processed 850/2000 images
[INFO] processed 900/2000 images
[INFO] processed 950/2000 images
[INFO] processed 1000/2000 images
[INFO] processed 1050/2000 images
[INFO] processed 1100/2000 images
[INFO] processed 1150/2000 images
[INFO] processed 1200/2000 images
[INFO] processed 1250/2000 images
[INFO] processed 1300/2000 images
[INFO] processed 1350/2000 images
[INFO] processed 1400/2000 images
[INFO] processed 1450/2000 images
[INFO] processed 1500/2000 images
